<a href="https://colab.research.google.com/github/kento-koyama/bayesian_predictive_micro_ICPMF12/blob/main/RJAGS_exapme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting for RJAGS

In [1]:
uname <- system("uname -a", intern = TRUE)
cat("System information:", uname, "\n")
# Get the current working directory
cwd <- getwd()
cat("Current working directory:", cwd, "\n")
system("sudo apt-get update")
system("sudo apt-get install jags")
# Check the JAGS location
jags_location <- system("which jags", intern = TRUE)
cat("JAGS Location:", jags_location, "\n")
temp_file <- system("mktemp", intern = TRUE)
# Download the JAGS Wiener module .deb file
jags_url <- "https://launchpad.net/~cidlab/+archive/ubuntu/jwm/+files/jags-wiener-module_1.1-5_amd64.deb"
wget_result <- system2("wget", c("-O", temp_file, jags_url), stdout = TRUE, stderr = TRUE)
wget_result
# Install the JAGS Wiener module .deb file
install_result <- system2("sudo", c("dpkg", "-i", temp_file), stdout = TRUE, stderr = TRUE)
# Remove the JAGS Wiener module .deb file
system(paste("rm -f", temp_file))
uname <- system("jags", intern = TRUE)
cat("System information:", uname, "\n")

System information: Linux b19d8456cbe0 5.10.147+ #1 SMP Sat Dec 10 16:00:40 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux 
Current working directory: /content 
JAGS Location: /usr/bin/jags 


[1] "--2023-04-13 11:57:33--  https://launchpad.net/~cidlab/+archive/ubuntu/jwm/+files/jags-wiener-module_1.1-5_amd64.deb"       
 [2] "Resolving launchpad.net (launchpad.net)... 185.125.189.222, 185.125.189.223, 2620:2d:4000:1001::8003, ..."                  
 [3] "Connecting to launchpad.net (launchpad.net)|185.125.189.222|:443... connected."                                             
 [4] "HTTP request sent, awaiting response... 303 See Other"                                                                      
 [5] "Location: https://launchpadlibrarian.net/289923160/jags-wiener-module_1.1-5_amd64.deb [following]"                          
 [6] "--2023-04-13 11:57:34--  https://launchpadlibrarian.net/289923160/jags-wiener-module_1.1-5_amd64.deb"                       
 [7] "Resolving launchpadlibrarian.net (launchpadlibrarian.net)... 185.125.189.229, 185.125.189.228, 2620:2d:4000:1001::8007, ..."
 [8] "Connecting to launchpadlibrarian.net (launchpadlibrarian.net)|185.125.189.229|:443... connected."                           
 [9] "HTTP request sent, awaiting response... 200 OK"                                                                             
[10] "Length: 18376 (18K) [application/x-debian-package]"                                                                         
[11] "Saving to: ‘/tmp/tmp.eRRE9nk2lv’"                                                                                           
[12] ""                                                                                                                           
[13] "     0K .......... .......                                    100% 63.1M=0s"                                                
[14] ""                                                                                                                           
[15] "2023-04-13 11:57:39 (63.1 MB/s) - ‘/tmp/tmp.eRRE9nk2lv’ saved [18376/18376]"                                                
[16] ""

System information: Welcome to JAGS 4.3.0 on Thu Apr 13 11:57:39 2023 JAGS is free software and comes with ABSOLUTELY NO WARRANTY Loading module: basemod: ok Loading module: bugs: ok .  


In [2]:
#Install and library
install.packages(c("rjags","runjags"))
library(rjags)
library(runjags)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘coda’


Loading required package: coda

Linked to JAGS 4.3.0

Loaded modules: basemod,bugs



# Bayesian modelling

In [3]:
n.occasions <- 50
a.tru <- -1
b.tru <- -2
c.tru <- 1
sd.tru <- 0.5

# coef に入れた任意の長さの数値ベクトルを係数として、多項式の値を出力する。
funcPolynom <- function(x, coef) {
    p <- length(coef)
    ans <- 0
    for (i in 1:p) {
        product <- coef[i]*(x^(i-1))
        ans <- ans + product
    }
    return(ans)
}

set.seed(114514) # 皆様のPC環境で同じ結果が得られるように、乱数のシードを指定。
x <- rnorm(n=n.occasions, mean=1, sd=1) # 説明変数の値を作成。
surv15 <- funcPolynom(x, coef=c(a.tru, b.tru, c.tru)) # 応答値 (mu) を作成。
surv15var <- surv15+rnorm(n.occasions, mean=0, sd=sd.tru) # 測定誤差の足された応答値 (y) を作成。


In [4]:
# 関数の形を描いておく
library(ggplot2)
XLIM <- c(-2, 4)
YLIM <- c(-4, 8)
filename.pdf <- "runjags_Fig01.pdf"
filename.png <- "runjags_Fig01.png"
p <- ggplot(data.frame(x=x), aes(x=x, y=surv15var)) + 
    stat_function(  fun=funcPolynom, 
                    args=list(coef=c(a.tru, b.tru, c.tru)), linetype=1, size=0.75) + 
    annotate("segment", x=x, xend=x, y=surv15, yend=surv15var, col=rgb(0, 0, 0), size=0.25) +
    geom_point(col=rgb(0, 0.5, 0.8), size=0.75) +
    geom_abline(intercept=0, slope=0, linetype="solid", size=1/3) + 
    theme_classic(base_size=9) +
    theme(legend.position="none") + # no legend
    coord_fixed(ratio=0.5) +
    scale_x_continuous(expand=c(0, 0), limit=XLIM, breaks=c(-2, 0, 2, 4)) + 
    scale_y_continuous(expand=c(0, 0), limit=YLIM, breaks=seq(-4, 8, by=2)) + 
    annotate(   "text", x=1, y=2, label="italic(mu) == -1 -2*italic(x) + italic(x)^2", 
                parse=TRUE, size=3, hjust=0.5) +
    labs( title="", x="x", y="y" )
ggsave(file=filename.pdf, device=cairo_pdf, plot=p, dpi=300, width=8/2.54, height=8/2.54, units="in")
ggsave(file=filename.png, plot=p, dpi=300, width=3, height=3, units="in")

Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”


In [5]:
# 観測データ
jags.data <- list(
  n = n.occasions,
  x = x,
  y = surv15var
)

# モデルファイル名
jagsname <- "model01.jags"

# モデルファイルの作成
writeLines("
model {
    Tau.nif <- 1.0E-3;
    P.gamma <- 1.0E-3;

    # 個々のパラメータに対する事前分布
    a ~ dnorm(0, Tau.nif);
    b ~ dnorm(0, Tau.nif);
    c ~ dnorm(0, Tau.nif);
    tau ~ dgamma(P.gamma, P.gamma);
    sigma <- 1/sqrt(tau);

    # 尤度モデル
    for (i in 1:n) {
        mu[i] <- a + b*x[i] + c*pow(x[i], 2); # x[i]^2 == pow(x[i], 2) in JAGS version 4
        y[i] ~ dnorm(mu[i], tau);
    }
}
", con = jagsname)


In [6]:

# MCMC の条件設定。
monitor <- c("a", "b", "c", "sigma") # 上のモデルで MC のサンプリング対象とする変数。
n.chains <- 3 # チェーンの数は、初期値リスト作成よりも前で定義すること


# stochastic node の初期値リスト。チェーンの数だけ用意する必要があるので、ジェネレータとして作る。
# .RNG.seed と .RNG.name に挟まれた行に、各 stochastic node の初期値を与える乱数を置く。
set.seed(10) # 後日結果を再現したい場合のため、乱数のシードを決めておく。
func_init <- function(i) {
    list(
        .RNG.seed=i+0.1,
        a=rnorm(1, 0, 1),
        b=rnorm(1, 0, 1),
        c=rnorm(1, 0, 1),
        tau=rgamma(1, 1, 1),
        .RNG.name="base::Mersenne-Twister")
}
inits <- list(func_init(1)) # 最初の要素を二重リストにしてから、for文内部で append する。
if (n.chains > 1) {
    for (i in 2:n.chains) {
        set.seed(100+i)
        inits <- append(inits, list(func_init(i))) # リストをさらにリストで包んでから連結。
    }
}
set.seed(1000);
str(inits); # 一応見ておきましょう


List of 3
 $ :List of 6
  ..$ .RNG.seed: num 1.1
  ..$ a        : num 0.0187
  ..$ b        : num -0.184
  ..$ c        : num -1.37
  ..$ tau      : num 0.166
  ..$ .RNG.name: chr "base::Mersenne-Twister"
 $ :List of 6
  ..$ .RNG.seed: num 2.1
  ..$ a        : num 0.181
  ..$ b        : num 0.785
  ..$ c        : num -1.35
  ..$ tau      : num 2.89
  ..$ .RNG.name: chr "base::Mersenne-Twister"
 $ :List of 6
  ..$ .RNG.seed: num 3.1
  ..$ a        : num -0.786
  ..$ b        : num 0.0547
  ..$ c        : num -1.17
  ..$ tau      : num 0.389
  ..$ .RNG.name: chr "base::Mersenne-Twister"


In [9]:
# MCMC の実行。複数コアを使うメソッド "parallel" で runjags する。
name_anl <- "runjags.test101"
library(rjags); library(runjags); findjags(); # パッケージをロード。おまじないで findjags() すべし。
system.time(
assign(name_anl, run.jags(  model=jagsname, monitor=monitor, data=jags.data,
                            inits=inits, n.chains=n.chains,
                            adapt=1000, burnin=49000, sample=10000, thin=10,# sample*thin 繰り返す
                            ) )
)
# Graph size 486, chain=3, system time 7.5 sec. # 150000回で7.5秒
get(name_anl) # 事後分布の表示
name_time <- format(Sys.time(), "%Y-%m-%d_%H.%M.%S")
plot(get(name_anl), layout=c(2, 2), file=paste(name_anl, name_time, "pdf", sep="."))
plot(get(name_anl), layout=c(2, 2))

[1] "/usr/bin/jags"

Compiling rjags model...
Calling the simulation using the rjags method...
Note: the model did not require adaptation
Burning in the model for 49000 iterations...
Running the model for 100000 iterations...
Simulation complete
Calculating summary statistics...
Calculating the Gelman-Rubin statistic for 4 variables....
Finished running the simulation


   user  system elapsed 
  5.243   0.036   5.368 


JAGS model summary statistics from 30000 samples (thin = 10; chains = 3; adapt+burnin = 50000):
                                                                             
      Lower95  Median  Upper95    Mean       SD Mode      MCerr MC%ofSD SSeff
a     -1.2811 -1.0756 -0.85861 -1.0752  0.10728   -- 0.00060837     0.6 31099
b     -2.1122 -1.8755  -1.6444 -1.8763  0.11842   -- 0.00074193     0.6 25476
c     0.85504 0.96271    1.069 0.96243  0.05428   -- 0.00034162     0.6 25246
sigma  0.4169 0.51279  0.62662 0.51749 0.054492   -- 0.00030764     0.6 31375
                        
          AC.100    psrf
a     0.00047752       1
b     -0.0042878  1.0001
c     -0.0012303 0.99998
sigma -0.0024568       1

Total time taken: 5.3 seconds


Generating plots...
Generating plots...
